### Imports

In [1]:
import pandas as pd
import numpy as np
import duckdb
import requests
from shapely.geometry import Point
import geopandas as gpd

### BiciMad Database

In [2]:
db_connect = duckdb.connect('../data/bicimad.db')

bicimad_connect = db_connect.execute('SELECT name AS "BiciMAD Station", address AS "Station Location",\
                                        "geometry.coordinates" AS "Coordinates", FROM main.bicimad_stations')

bicimad_df1 = bicimad_connect.fetchdf()

In [89]:
columns = bicimad_df1['Coordinates'].str.strip('[]').str.split(pat = ',', expand = True, regex=False)

bicimad_df1[['Longitude', 'Latitude']] = columns

bicimad_df1 = bicimad_df1.astype({'Longitude':'float','Latitude':'float'})

bicimad_df = bicimad_df1.drop('Coordinates', axis=1)

bicimad_df = bicimad_df.reindex(columns=['BiciMAD Station','Station Location','Latitude','Longitude'])

bicimad_df

,BiciMAD Station,Station Location,Latitude,Longitude
0,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834
1,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603
2,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917
4,4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587
...,...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,40.438960,-3.729970
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295


### Place of Interest Data

In [4]:
# Endpoint

url1 = 'https://datos.madrid.es/egob'
url2 = '/catalogo/209426-0-templos-catolicas.json'

In [5]:
response = requests.get(url1 + url2)

json_data = response.json()

df = pd.json_normalize(json_data['@graph'])

In [54]:
places_df1 = df[['title','@type','address.street-address','location.latitude','location.longitude']]

places_df2 = places_df1.rename(columns={'title': 'Place of Interest','@type':'Type of Place',\
                                        'address.street-address':'Place Address',\
                                       'location.latitude':'Latitude','location.longitude':'Longitude'})

places_df2['Type of Place'] = places_df2['Type of Place'].str.extract('/entidadesYorganismos/(\w+)')
places_df2['Type of Place'] = places_df2['Type of Place'].str.findall('[A-Z][^A-Z]*').str.join(' ')
places_df2['Place Address'] = places_df2['Place Address'].str.title()
places_df2=places_df2.loc[places_df2['Type of Place'] == "Templos Iglesias Catolicas"]
places_df2

,Place of Interest,Type of Place,Place Address,Latitude,Longitude
0,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608
1,Basílica La Milagrosa,Templos Iglesias Catolicas,Calle Garcia De Paredes 45,40.436738,-3.695725
3,Capilla de la Virgen de Nuria,Templos Iglesias Catolicas,Calle Juan De Urbieta 5,40.403887,-3.678216
4,Capilla de San Ignacio de los Vascos,Templos Iglesias Catolicas,Calle Principe 31,40.414334,-3.700102
6,Capilla Nuestra Señora de la Bien Aparecida,Templos Iglesias Catolicas,Calle Pio Baroja 2,40.415285,-3.674697
...,...,...,...,...,...
382,Santuario Nuestra Señora de los Peligros,Templos Iglesias Catolicas,Calle Joaquin Costa 49,40.442252,-3.682151
383,Santuario Nuestra Señora de Schönstatt,Templos Iglesias Catolicas,Calle Serrano 97,40.438298,-3.686450
384,Segundo Monasterio de la Visitación (Las Salesas),Templos Iglesias Catolicas,Calle San Bernardo 72,40.427468,-3.706204
385,Templo Eucarístico de San Martín de Tours,Templos Iglesias Catolicas,Calle Desenga&Amp;Ntilde;O 26,40.421207,-3.704641


### Functions

In [7]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [55]:
combined_df = places_df2.merge(bicimad_df, how='cross')
combined_df

,Place of Interest,Type of Place,Place Address,Latitude_x,Longitude_x,BiciMAD Station,Station Location,Latitude_y,Longitude_y
0,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834
1,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603
2,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842
3,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917
4,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587
...,...,...,...,...,...,...,...,...,...
94243,Tercer Monasterio de la Visitación (Servidoras...,Templos Iglesias Catolicas,Paseo San Francisco De Sales 48,40.443602,-3.710745,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,40.438960,-3.729970
94244,Tercer Monasterio de la Visitación (Servidoras...,Templos Iglesias Catolicas,Paseo San Francisco De Sales 48,40.443602,-3.710745,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990
94245,Tercer Monasterio de la Visitación (Servidoras...,Templos Iglesias Catolicas,Paseo San Francisco De Sales 48,40.443602,-3.710745,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930
94246,Tercer Monasterio de la Visitación (Servidoras...,Templos Iglesias Catolicas,Paseo San Francisco De Sales 48,40.443602,-3.710745,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295


In [56]:
# Prueba con una parte del Dataframe
combined_df_loc = combined_df.iloc[:10,:]
combined_df_loc

,Place of Interest,Type of Place,Place Address,Latitude_x,Longitude_x,BiciMAD Station,Station Location,Latitude_y,Longitude_y
0,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834
1,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603
2,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842
3,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917
4,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587
5,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,5 - Fuencarral,Calle Fuencarral nº 108,40.428520,-3.702050
6,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,6 - Colegio Arquitectos,Calle Hortaleza nº 63,40.424148,-3.698447
7,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,7 - Hortaleza,Calle Hortaleza nº 75,40.425191,-3.697771
8,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,8 - Alonso Martínez,Plaza de Alonso Martínez nº 5,40.427868,-3.695440
9,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,9 - Plaza de San Miguel,Plaza de San Miguel nº 9,40.415606,-3.709508


In [57]:
%time
combined_df_loc['Distance'] = combined_df_loc.apply(lambda x: distance_meters(x['Latitude_x'], x['Longitude_x'],\
                                            x['Latitude_y'], x['Longitude_y']), axis=1)
combined_df_loc

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 9.06 µs


/var/folders/19/xp05rklx1dsfj3fpvkq0qry00000gn/T/ipykernel_8546/3379781229.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_loc['Distance'] = combined_df_loc.apply(lambda x: distance_meters(x['Latitude_x'], x['Longitude_x'],\


,Place of Interest,Type of Place,Place Address,Latitude_x,Longitude_x,BiciMAD Station,Station Location,Latitude_y,Longitude_y,Distance
0,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,780.283863
1,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603,762.661891
2,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842,1355.736981
3,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917,2207.690979
4,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587,1795.491308
5,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,5 - Fuencarral,Calle Fuencarral nº 108,40.428520,-3.702050,1767.021633
6,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,6 - Colegio Arquitectos,Calle Hortaleza nº 63,40.424148,-3.698447,1171.243672
7,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,7 - Hortaleza,Calle Hortaleza nº 75,40.425191,-3.697771,1266.880879
8,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,8 - Alonso Martínez,Plaza de Alonso Martínez nº 5,40.427868,-3.695440,1541.871795
9,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,9 - Plaza de San Miguel,Plaza de San Miguel nº 9,40.415606,-3.709508,1560.615733


In [58]:
%time
combined_df['Distance'] = combined_df.apply(lambda x: distance_meters(x['Latitude_x'], x['Longitude_x'],\
                                            x['Latitude_y'], x['Longitude_y']), axis=1)
combined_df

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.72 µs


,Place of Interest,Type of Place,Place Address,Latitude_x,Longitude_x,BiciMAD Station,Station Location,Latitude_y,Longitude_y,Distance
0,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,780.283863
1,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603,762.661891
2,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842,1355.736981
3,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917,2207.690979
4,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,40.414018,-3.695608,4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587,1795.491308
...,...,...,...,...,...,...,...,...,...,...
94243,Tercer Monasterio de la Visitación (Servidoras...,Templos Iglesias Catolicas,Paseo San Francisco De Sales 48,40.443602,-3.710745,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,40.438960,-3.729970,2206.027766
94244,Tercer Monasterio de la Visitación (Servidoras...,Templos Iglesias Catolicas,Paseo San Francisco De Sales 48,40.443602,-3.710745,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990,1812.285405
94245,Tercer Monasterio de la Visitación (Servidoras...,Templos Iglesias Catolicas,Paseo San Francisco De Sales 48,40.443602,-3.710745,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930,1805.631344
94246,Tercer Monasterio de la Visitación (Servidoras...,Templos Iglesias Catolicas,Paseo San Francisco De Sales 48,40.443602,-3.710745,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295,1919.797432


In [83]:
# Luego debo hacer un GroupBy para encontrar los lugares con la distancia mas corta

final_df = combined_df.loc[combined_df.groupby("Place of Interest")["Distance"].idxmin()]
final_df = final_df.drop(['Latitude_x','Longitude_x','Latitude_y','Longitude_y'], axis=1)
final_df = final_df.reset_index(drop=True)
final_df['Distance'] = final_df['Distance'].round(0)
final_df

,Place of Interest,Type of Place,Place Address,BiciMAD Station,Station Location,Distance
0,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,27 - Huertas,Calle Jesús nº 1,82.0
1,Basílica La Milagrosa,Templos Iglesias Catolicas,Calle Garcia De Paredes 45,125 - General Martínez Campos,Calle Fernández de la Hoz nº 29,188.0
2,Capilla Nuestra Señora de la Bien Aparecida,Templos Iglesias Catolicas,Calle Pio Baroja 2,71 - Doce de Octubre,Calle Doce de Octubre nº 28,117.0
3,Capilla de San Ignacio de los Vascos,Templos Iglesias Catolicas,Calle Principe 31,52 - Plaza de Santa Ana,Plaza de Santa Ana nº 10,69.0
4,Capilla de la Virgen de Nuria,Templos Iglesias Catolicas,Calle Juan De Urbieta 5,178 - Junta Municipal Retiro,Calle León Gil de Palacio nº 3,128.0
...,...,...,...,...,...,...
352,Santuario Nuestra Señora de Schönstatt,Templos Iglesias Catolicas,Calle Serrano 97,250 - Serrano 113,Calle Serrano nº 113B,310.0
353,Santuario Nuestra Señora de los Peligros,Templos Iglesias Catolicas,Calle Joaquin Costa 49,140 - Velázquez 130,Calle Velázquez nº 130,431.0
354,Segundo Monasterio de la Visitación (Las Salesas),Templos Iglesias Catolicas,Calle San Bernardo 72,12 - San Hermenegildo,Calle San Bernardo nº 85,107.0
355,Templo Eucarístico de San Martín de Tours,Templos Iglesias Catolicas,Calle Desenga&Amp;Ntilde;O 26,2 - Miguel Moya,Calle Miguel Moya nº 1,151.0


In [85]:
csv_df = final_df[['Place of Interest', 'Type of Place','Place Address','BiciMAD Station','Station Location']]

csv_df.to_csv('Main Challenge.csv', index=False)